In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


# Building a Search Engine with BERT and TensorFlow


Feature extractors based on Neural Probabilistic Language Models [http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf]  such as BERT [https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270] extract features that are relevant for a wide array of downstream NLP tasks. For that reason, they are sometimes referred to as Natural Language Understanding (NLU) [https://en.wikipedia.org/wiki/Natural-language_understanding] modules.

These features may also be used for instance-based learning [https://en.wikipedia.org/wiki/Instance-based_learning] which relies on computing the similarity of the query to the training samples. To demonstrate this, we will use the BERT NLU module for feature extraction to build a nearest neighbour search engine.

Now, let’s start.

## Step 1: getting the pre-trained model

We start with a pre-trained BERT checkpoint. For demonstration purposes, we will be using the uncased English model [https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip] pre-trained by Google engineers.

For configuring and optimizing the graph for inference we will make use of the awesome bert-as-a-service [https://github.com/hanxiao/bert-as-service] repository. This repository allows for serving BERT models for remote clients over TCP.

Having a remote BERT-server is beneficial in multi-host environments. However, in this part of the experiment we will focus on creating a local 
(in-process) feature extractor. This is useful if one wishes to avoid additional latency and potential failure modes introduced by a client-server architecture.

Now, let us download the model and install the package.

```
!wget https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
!unzip wwm_uncased_L-24_H-1024_A-16.zip
!pip install bert-serving-server --no-deps
```

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip -P /gdrive/My\ Drive/Colab\ Notebooks/datasets

--2019-06-30 14:28:27--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘/gdrive/My Drive/Colab Notebooks/datasets/uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  64.2MB/s    in 6.2s    

2019-06-30 14:28:34 (62.8 MB/s) - ‘/gdrive/My Drive/Colab Notebooks/datasets/uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [0]:
!unzip uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [0]:
!pip install bert-serving-server --no-deps

## Step 2: optimizing the inference graph

Normally, to modify the model graph we would have to do some low-level TensorFlow programming. However, thanks to bert-as-a-service, we can configure the inference graph using a simple CLI interface.

In [0]:
import os
import tensorflow as tf

In [0]:
from bert_serving.server.graph import optimize_graph
from bert_serving.server.helper import get_args_parser

In [0]:
MODEL_DIR = '/content/uncased_L-12_H-768_A-12/'
GRAPH_DIR = '/content/graph'
GRAPH_OUT = 'extractor.pbtxt'
GPU_MFRAC = 0.2

In [0]:
POOL_START = 'REDUCE_MEAN'
POOL_LAYER = '-2'
SEQ_LEN = '64'

In [0]:
tf.gfile.MkDir(GRAPH_DIR)

In [0]:
parser = get_args_parser()
carg = parser.parse_args(args=[
    '-model_dir', MODEL_DIR,
    '-graph_tmp_dir', GRAPH_DIR,
    '-max_seq_len', str(SEQ_LEN),
    '-pooling_layer', str(POOL_LAYER),
    '-pooling_strategy', str(POOL_START),
    '-gpu_memory_fraction', str(GPU_MFRAC)
])

In [0]:
tmfi_name, config = optimize_graph(carg)

W0630 17:41:24.834964 140043474638720 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert_serving/server/helper.py:178: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0630 17:41:24.837943 140043474638720 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert_serving/server/helper.py:178: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.

I:GRAPHOPT:[gra:opt: 52]:model config: /content/uncased_L-12_H-768_A-12/bert_config.json
I0630 17:41:24.841775 140043474638720 graph.py:52] model config: /content/uncased_L-12_H-768_A-12/bert_config.json
I:GRAPHOPT:[gra:opt: 55]:checkpoint: /content/uncased_L-12_H-768_A-12/bert_model.ckpt
I0630 17:41:24.844499 140043474638720 graph.py:55] checkpoint: /content/uncased_L-12_H-768_A-12/bert_model.ckpt
I:GRAPHOPT:[gra:opt: 59]:build graph...
I0630 17:41:24.847904 140043474638720 graph.py:59] build graph...
I:GRAPHO

In [0]:
graph_fout = os.path.join(GRAPH_DIR, GRAPH_OUT)

In [0]:
tf.gfile.Rename(tmfi_name, graph_fout, overwrite=True)
print("Serialized graph to {}". format(graph_fout))

Serialized graph to /content/graph/extractor.pbtxt


There are a couple of parameters there to look out for.

For each text sample, BERT encoding layers output a tensor of shape [*sequence_len, encoder_dim*] with one vector per token. We need to apply some sort of pooling if we are to obtain a fixed representation.

**POOL_STRAT** parameter defines the pooling strategy applied to the encoding layer number **POOL_LAYER**. The default value ‘*REDUCE_MEAN*’ averages the vectors for all tokens in a sequence. This strategy works best for most sentence-level tasks when the model is not fine-tuned. Another option is *NONE*, in which case no pooling is applied at all. This is useful for word-level tasks such as Named Entity Recognition or POS tagging. For a detailed discussion of these options check out the Han Xiao’s blog post [https://hanxiao.github.io/2019/01/02/Serving-Google-BERT-in-Production-using-Tensorflow-and-ZeroMQ/].

**SEQ_LEN** affects the maximum length of sequences processed by the model. Smaller values will increase the model inference speed almost linearly.

Running the above command will put the model graph and weights into a GraphDef [https://www.tensorflow.org/guide/extend/model_files] object which will be serialized to a pbtxt file at **GRAPH_OUT**. The file will often be smaller than the pre-trained model because the nodes and variables required for training will be removed. This results in a very portable solution: for example, the english model only takes 380 MB after serializing.

## Step 3: Creating a feature extractor

Now, we will use the serialized graph to build a feature extractor using the tf.Estimator API. We will need to define two things: **input_fn** and **model_fn**

**input_fn** manages getting the data into the model. That includes executing the whole text preprocessing pipeline and preparing a **feed_dict** for BERT.

First, each text sample is converted into a **tf.Example** instance containing the necessary features listed in INPUT_NAMES. The **bert_tokenizer** object contains the WordPiece vocabulary and performs the text preprocessing. After that, the examples are re-grouped by feature name in a **feed_dict**.

In [0]:
INPUT_NAMES = ['input_ids', 'input_mask', 'input_type_ids']
VOCAB_PATH = MODEL_DIR + "vocab.txt"

SEQ_LEN = 256

In [0]:
import logging

log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)
log.handlers = []

In [0]:
from bert_serving.server.bert.tokenization import FullTokenizer
from bert_serving.server.bert.extract_features import convert_lst_to_features

bert_tokenizer = FullTokenizer(VOCAB_PATH)

In [0]:
import numpy as np

def build_feed_dict(texts):
  
  text_features = list(convert_lst_to_features(
      texts, SEQ_LEN, SEQ_LEN,
      bert_tokenizer, log, False, False
  ))
  
  target_shape = (len(texts), -1)
  
  feed_dict = {}
  for iname in INPUT_NAMES:
    features_i = np.array([getattr(f, iname) for f in text_features])
    features_i = features_i.reshape(target_shape)
    features_i = features_i.astype("int32")
    feed_dict[iname] = features_i
    
  return feed_dict

tf.Estimators have a fun feature which makes them rebuild and reinitialize the whole computational graph at each call to the predict function. So, in order to avoid the overhead, we will pass a generator to the predict function, and the generator will yield the features to the model in a never-ending loop.

In [0]:
def build_input_fn(container):
  
  def gen():
    while True:
      try:
        yield build_feed_dict(container.get())
      except:
        yield build_feed_dict(container.get())
      
  
  def input_fn():
    return tf.data.Dataset.from_generator(
        gen,
        output_types={iname: tf.int32 for iname in INPUT_NAMES},
        output_shapes={iname: (None, None) for iname in INPUT_NAMES}
    )
  
  return input_fn

In [0]:
class DataContainer:
  def __init__(self):
    self._texts = None
    
  def set(self, texts):
    if type(texts) is str:
      texts = [texts]
    self._texts = texts
    
  def get(self):
    return self._texts

**model_fn** contains the specification of the model. In our case, it is loaded from the pbtxt file we saved in the previous step. The features are mapped explicitly to the corresponding input nodes via **input_map**.

In [0]:
from tensorflow.python.estimator.model_fn import EstimatorSpec
from tensorflow.python.estimator.estimator import Estimator

GRAPH_PATH = os.path.join(GRAPH_DIR, GRAPH_OUT)

def model_fn(features, mode):
  with tf.gfile.GFile(GRAPH_PATH, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    
  output = tf.import_graph_def(graph_def,
                                 input_map={k + ':0': features[k] for k in INPUT_NAMES},
                                 return_elements=['final_encodes:0'])
  
  return EstimatorSpec(mode=mode, predictions={'output': output[0]})

Now we have almost everything we need to perform inference. Let’s do this!

In [0]:
def batch(iterable,n=1):
  l = len(iterable)
  for ndx in range(0, 1, n):
    yield iterable[ndx:min(ndx + n, 1)]

In [0]:
from tensorflow.keras.utils import Progbar

def build_vectorizer(_estimator, _input_fn_builder, batch_size=128):
  container = DataContainer()
  predict_fn = _estimator.predict(_input_fn_builder(container), yield_single_examples=False)
  
  def vectorize(text, verbose=False):
    x = []
    bar = Progbar(len(text))
    for text_batch in batch(text, batch_size):
      container.set(text_batch)
      x.append(next(predict_fn)['output'])
      if verbose:
        bar.add(len(text_batch))
        
    r = np.vstack(x)
    return r
  
  return vectorize

In [0]:
estimator = Estimator(model_fn=model_fn)
bert_vectorizer = build_vectorizer(estimator, build_input_fn)
# bert_vectorizer

I0630 17:45:02.072189 140043474638720 estimator.py:1790] Using default config.
W0630 17:45:02.076397 140043474638720 estimator.py:1811] Using temporary folder as model directory: /tmp/tmpznh5gnyc
I0630 17:45:02.077595 140043474638720 estimator.py:209] Using config: {'_model_dir': '/tmp/tmpznh5gnyc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5e07002710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master

In [0]:
bert_vectorizer(64 * ['sample text']).shape

I0630 17:45:04.441466 140043474638720 estimator.py:612] Could not find trained model in model_dir: /tmp/tmpznh5gnyc, running initialization to predict.
I0630 17:45:04.474352 140043474638720 estimator.py:1145] Calling model_fn.
I0630 17:45:06.002777 140043474638720 estimator.py:1147] Done calling model_fn.
I0630 17:45:06.067094 140043474638720 monitored_session.py:240] Graph was finalized.
I0630 17:45:06.115818 140043474638720 session_manager.py:500] Running local_init_op.
I0630 17:45:06.126268 140043474638720 session_manager.py:502] Done running local_init_op.


(1, 768)

## Step 4: exploring vector space with Projector


Now it’s time for a demonstration!

Using the vectorizer, we will generate embeddings for articles from the Reuters-21578 benchmark corpus.

To visualize and explore the embedding vector space in 3D, we will use a dimensionality reduction technique called T-SNE [https://distill.pub/2016/misread-tsne/].

Let’s get the article embeddings first.

In [0]:
import nltk
from nltk.corpus import reuters

nltk.download("reuters")
nltk.download("punkt")

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
max_samples = 256
categories = ['wheat', 'tea', 'strategic-metal', 'housing', 'money-supply', 'fuel']

S, X, Y = [], [], []

In [0]:
for category in categories:
  print(category)
  
  sents = reuters.sents(categories=category)
  sents = [' '.join(sent) for sent in sents][:max_samples]
  X.append(bert_vectorizer(sents, verbose=True))
  Y += [category] * len(sents)
  S += sents

wheat
  1/129 [..............................] - ETA: 1:57

In [0]:
X = np.vstack(X) 
X.shape

(6, 768)

The interactive visualization of generated embeddings is available on the Embedding Projector [https://projector.tensorflow.org/?config=https://gist.githubusercontent.com/gaphex/7262af1e151957b1e7c638f4922dfe57/raw/3b946229fc58cbefbca2a642502cf51d4f8e81c5/reuters_proj_config.json] .

From the link we can run T-SNE ourself, or load a checkpoint using the bookmark in lower-right corner (loading works only on Chrome).

To reproduce the input files used for this visualization, run the code below. Then, download the files to our machine and upload to Projector

(we can dowload files from the menu opened by the ">" button in the upper-left)

In [0]:
with open("embeddings.tsv", "w") as fo:
  for x in X.astype('float16'):
    line = "\t".join([str(v) for v in x])
    fo.write(line + "\n")

In [0]:
with open("metadata.tsv", "w") as fo:
  fo.write("Label\tSentence\n")
  for y, s in zip(Y, S):
    fo.write("{}\t{}\n".format(y, s))

Here's what we captured using the Projector.

In [0]:
from IPython.display import HTML

HTML("""
<video width="900" height="632" controls>
  <source src="https://storage.googleapis.com/bert_resourses/reuters_tsne_hd.mp4" type="video/mp4">
</video>
""")

## Step 5: building a nearest neighbour search engine

Now, let’s say we have a knowledge base of 50k text samples and we need to answer queries based on this data, fast. How do we retrieve the sample most similar to a query from a text database? The answer is nearest neighbour search [https://en.wikipedia.org/wiki/Nearest_neighbor_search].

Formally, the search problem we will be solving is defined as follows:

given a set of points S in vector space M, and a query point Q ∈ M, find the closest point in S to Q.

There are multiple ways to define 'closest' in vector space, but we will use Euclidean distance.

So, to build an Information Retrieval system, we will follow these steps:

1. Vectorize all samples from the knowledge base - that gives S
2. Vectorize the query - that gives Q
3. Compute euclidean distances D between Q and S
4. Sort D in ascending order , providing indices of the most similar samples
5. Retrieve labels for said samples from the knowledge base


To make the simple matter of implementing this a bit more exciting, we will do it in pure TensorFlow.

First we define the placeholders for Q and S

In [0]:
graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
dim = X.shape[1]

Q = tf.placeholder("float", [dim])
S = tf.placeholder("float", [None, dim])


Define distance computation

In [0]:
squared_distance = tf.reduce_sum(tf.pow(Q - S, 2), reduction_indices=1)
distance = tf.sqrt(squared_distance)

Finally, get the most similar indices

In [0]:
top_k = 10

top_neg_dists, top_indices = tf.math.top_k(tf.negative(distance), k=top_k)
top_dists = tf.negative(top_neg_dists)

## Step 6: accelerating search with math

Now that we have a basic retrieval engine set up, the question is: can we make it run faster? With a tiny bit of math, we can.

For a pair of vectors p and q the euclidean distance is defined as follows:

$$
d(p, q) = d(q, p) = \sqrt{(p_1 - q_1)^2 + (p_2 - q_2)^2+\cdots + (p_n - q_n)^2} =  \sqrt{\sum_{i=1}^n (p_i - q_i)^2}
$$

which is exactly how we did compute it in step 4.

However, since p and q are vectors, we can expand and rewrite this:

$$
d(p, q)^2 = \langle p - q, q - p \rangle = \langle pp \rangle + 2 \langle pq \rangle + \langle qq \rangle
$$

where ⟨…⟩ denotes inner product.

In TensorFlow this can be written as follows:

In [0]:
Q = tf.placeholder("float", [dim])
S = tf.placeholder("float", [None, dim])

In [0]:
Qr = tf.reshape(Q, (1, -1))

In [0]:
PP = tf.keras.backend.batch_dot(S, S, axes=1)
QQ = tf.matmul(Qr, tf.transpose(Qr))
PQ = tf.matmul(S, tf.transpose(Qr))

In [0]:
distance = PP - 2 * PQ + QQ
distance = tf.sqrt(tf.reshape(distance, (-1,)))

In [0]:
top_neg_dists, top_indices = tf.math.top_k(tf.negative(distance), k=top_k)

Due to the fact that matrix multiplication op is highly optimized, this implementation works slightly faster than the previous one.

By the way, in the formula above **PP** and **QQ** are actually squared L2 norms [https://en.wikipedia.org/wiki/Norm_%28mathematics%29#Euclidean_norm] of the respective vectors. If both vectors are L2 normalized, then **PP = QQ = 1**. That gives an interesting relation between inner product and euclidean distance:

$$
d(p, q)^2 = 2 \times (1 - \langle pq \rangle)
$$

However, doing L2 normalization discards the information about vector magnitude which in many cases is undesirable .

Instead, we may notice that as long as the knowledge base does not change, **PP**, its squared vector norm, also remains constant. So, instead of recomputing it every time we can just do it once and then use the precomputed result further accelerating the distance computation.

Now let us put it all together.

In [0]:
class L2Retriever:
    def __init__(self, dim, top_k=3, use_norm=False, use_gpu=True):
        self.dim = dim
        self.top_k = top_k
        self.use_norm = use_norm
        config = tf.ConfigProto(
            device_count={'GPU': (1 if use_gpu else 0)}
        )
        config.gpu_options.allow_growth = True
        self.session = tf.Session(config=config)
        
        self.norm = None
        self.query = tf.placeholder("float", [self.dim])
        self.kbase = tf.placeholder("float", [None, self.dim])
        
        self.build_graph()

    def build_graph(self):
      
        if self.use_norm:
            self.norm = tf.placeholder("float", [None, 1])

        distance = dot_l2_distances(self.kbase, self.query, self.norm)
        top_neg_dists, top_indices = tf.math.top_k(tf.negative(distance), k=self.top_k)
        top_dists = tf.negative(top_neg_dists)

        self.top_distances = top_dists
        self.top_indices = top_indices

    def predict(self, kbase, query, norm=None):
        query = np.squeeze(query)
        feed_dict = {self.query: query, self.kbase: kbase}
        if self.use_norm:
          feed_dict[self.norm] = norm
        
        I, D = self.session.run([self.top_indices, self.top_distances],
                                feed_dict=feed_dict)
        
        return I, D
      
def dot_l2_distances(kbase, query, norm=None):
    query = tf.reshape(query, (1, -1))
    
    if norm is None:
      XX = tf.keras.backend.batch_dot(kbase, kbase, axes=1)
    else:
      XX = norm
    YY = tf.matmul(query, tf.transpose(query))
    XY = tf.matmul(kbase, tf.transpose(query))
    
    distance = XX - 2 * XY + YY
    distance = tf.sqrt(tf.reshape(distance, (-1,)))
    
    return distance


# Example: movie recommendation system

For this example we will use a dataset of movie summaries from IMDB. Using the NLU and Retriever modules, we will build a movie recommendation system that suggests movies with similar plot features.

First, let’s download and prepare the IMDB dataset [http://www.cs.cmu.edu/~ark/personas/].

In [0]:
import pandas as pd
import json

In [0]:
!wget https://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz
!tar -xvzf MovieSummaries.tar.gz

--2019-06-30 18:20:54--  https://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48002242 (46M) [application/x-tar]
Saving to: ‘MovieSummaries.tar.gz’

MovieSummaries.tar. 100%[===================>]  45.78M   913KB/s    in 52s     

2019-06-30 18:21:47 (900 KB/s) - ‘MovieSummaries.tar.gz’ saved [48002242/48002242]

MovieSummaries/
MovieSummaries/tvtropes.clusters.txt
MovieSummaries/name.clusters.txt
MovieSummaries/plot_summaries.txt
MovieSummaries/README.txt
MovieSummaries/movie.metadata.tsv
MovieSummaries/character.metadata.tsv


In [0]:
plots_df = pd.read_csv('MovieSummaries/plot_summaries.txt', sep='\t', header=None)
meta_df = pd.read_csv('MovieSummaries/movie.metadata.tsv', sep='\t', header=None)

In [0]:
plots_df.tail()

,0,1
42298,34808485,"The story is about Reema , a young Muslim scho..."
42299,1096473,"In 1928 Hollywood, director Leo Andreyev look..."
42300,35102018,American Luthier focuses on Randy Parsons’ tra...
42301,8628195,"Abdur Rehman Khan , a middle-aged dry fruit se..."
42302,6040782,1940 - Operation Dynamo has just taken place. ...


In [0]:
meta_df.tail()

,0,1,2,3,4,5,6,7,8
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}"
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0..."
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}"
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ..."
81740,12476867,/m/02w7zz8,Spliced,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0d060g"": ""Canada""}","{""/m/01jfsb"": ""Thriller"", ""/m/03npn"": ""Horror""..."


In [0]:
plot = {}
metadata = {}
movie_data = {}

In [0]:
for movie_id, movie_plot in plots_df.values:
  plot[movie_id] = movie_plot 

In [0]:
for movie_id, movie_name, movie_genre in meta_df[[0,2,8]].values:
  genre = list(json.loads(movie_genre).values())
  if len(genre):
    metadata[movie_id] = {"name": movie_name,
                          "genre": genre}

In [0]:
for movie_id in set(plot.keys()) & set(metadata.keys()):
  movie_data[metadata[movie_id]['name']] = {"genre": metadata[movie_id]['genre'],
                                            "plot": plot[movie_id]}

In [0]:
movie_data

{'The Crypt': {'genre': ['Horror'],
  'plot': '{{Plot}} The movie starts off in 1979 as a man breaks into a closed-off crypt. The grave robber sneaks into a crypt and immediately searches through a coffin for valuables. He nonchalantly removes and steals a ring from the decaying finger of the skeleton in the tomb. He reaches inside once more, but an unknown force slams the coffin\'s cover closed, cutting off one of his hands in the process. In terror, the robber watches as the exit of the crypt is being closed off, trapping him. It then switches to present day, where a man with the name of Perry is being released from prison. His girlfriend, PJ, waits for him in a car nearby. Three other girls, Dorian, Adriana, and Morgan, are on their way to go meet them, but need to steal a car. They are driving away when the real owner of the stolen car shoots at them. They cheer as they drive away unscathed, only to realize that Adriana\'s brother was shot in the head and killed. Although hysterica

In [0]:
X, Y, names = [], [], []

for movie_name, movie_meta in movie_data.items():
  X.append(movie_meta['plot'])
  Y.append(movie_meta['genre'])
  names.append(movie_name)

Vectorize movie plots with the BERT NLU module. This will take a while.

In [0]:
X

['{{Plot}} The movie starts off in 1979 as a man breaks into a closed-off crypt. The grave robber sneaks into a crypt and immediately searches through a coffin for valuables. He nonchalantly removes and steals a ring from the decaying finger of the skeleton in the tomb. He reaches inside once more, but an unknown force slams the coffin\'s cover closed, cutting off one of his hands in the process. In terror, the robber watches as the exit of the crypt is being closed off, trapping him. It then switches to present day, where a man with the name of Perry is being released from prison. His girlfriend, PJ, waits for him in a car nearby. Three other girls, Dorian, Adriana, and Morgan, are on their way to go meet them, but need to steal a car. They are driving away when the real owner of the stolen car shoots at them. They cheer as they drive away unscathed, only to realize that Adriana\'s brother was shot in the head and killed. Although hysterical, they continue to drive to meet the others.

In [0]:
bert_vectorizer = build_vectorizer(estimator, build_input_fn)
X_vect = bert_vectorizer(X, verbose=True)

Exception ignored in: <generator object gen at 0x7f5df3b1e3b8>
RuntimeError: generator ignored GeneratorExit
I0630 18:36:25.418301 140043474638720 estimator.py:612] Could not find trained model in model_dir: /tmp/tmpznh5gnyc, running initialization to predict.
I0630 18:36:25.450896 140043474638720 estimator.py:1145] Calling model_fn.
I0630 18:36:26.860064 140043474638720 estimator.py:1147] Done calling model_fn.
I0630 18:36:26.920960 140043474638720 monitored_session.py:240] Graph was finalized.
I0630 18:36:26.967945 140043474638720 session_manager.py:500] Running local_init_op.
I0630 18:36:26.978907 140043474638720 session_manager.py:502] Done running local_init_op.


    1/39526 [..............................] - ETA: 48:48:43

In [0]:
X_square_norm = np.sum(X_vect**2, axis=1).reshape((-1,1))

Finally, using the L2Retriever, find movies with plot vectors most similar to the query movie, and return it to user.

In [0]:
def buildMovieRecommender(movie_names, vectorized_plots, top_k=10):
  retriever = L2Retriever(vectorized_plots.shape[1], use_norm=True, top_k=top_k, use_gpu=True)
  vectorized_norm = np.sum(vectorized_plots**2, axis=1).reshape((-1,1))
  
  def recommend(query):
    try:
      idx = retriever.predict(vectorized_plots, 
                              vectorized_plots[movie_names.index(query)], 
                              vectorized_norm)[0][1:]
      for i in idx:
        print(names[i])
    except ValueError:
      print("{} not found in movie db. Suggestions:")
      for i, name in enumerate(movie_names):
        if query.lower() in name.lower():
          print(i, name)
          
  return recommend

Let’s check it out!

In [0]:
recommend = buildMovieRecommender(names, X_vect)
recommend("The Matrix")

RuntimeError: ignored

Even without supervision, the model performs adequately on several classification and retrieval tasks. While the performance can be further improved with supervised data, the described approach to text feature extraction provides a solid baseline for downstream NLP solutions.

This concludes the guide to building a search engine with BERT and TensorFlow.